In [1]:
from source.regency import Regency
from source.DQNAgent import DQNAgent
from IPython.display import clear_output
import warnings
warnings.filterwarnings('ignore')


Game = Regency()
clear_output()
for a in range(10):
    Game.random_events()
    Game.collect_regency_points()
    Game.domain_initiative()
    Game.collect_gold_bars()
    Game.maintenance_costs(Update=True)
    Game.take_domain_actions(None)
Game.clear_screen()
Game.Seasons[Game.Season]['Season']

,Regent,Player,Random Event,Regency Points,Initiative,Gold Bars,Revenue,Cost,Court
0,Ca,False,No Event,200,20,0,0,0,Dormant
1,Is,False,No Event,1044,20,25,9,13,Rich
2,AV,False,No Event,40,19,3,3,2,Dormant
3,GH,False,No Event,455,19,237,39,12,Rich
4,GG,False,Corruption,421,19,-5,34,39,Dormant
5,GTh,False,No Event,74,18,4,7,7,Bare
6,EL,False,No Event,113,18,16,7,8,Average
7,PAI,False,Assassination,348,17,155,27,13,Rich
8,JR,False,Festival,322,16,13,11,8,Average
9,SM,False,No Event,150,16,0,0,0,Dormant


In [21]:
Game.Regents[Game.Regents['Alive']==False]

,Regent,Full Name,Player,Class,Level,Alignment,Race,Str,Dex,Con,...,Wis,Cha,Insight,Deception,Persuasion,Regency Points,Gold Bars,Regency Bonus,Attitude,Alive


In [22]:
Game.Provences[Game.Provences['Castle']>0]

,Provence,Domain,Region,Regent,Terrain,Loyalty,Taxation,Population,Magic,Castle,Castle Name,Capital,Position,Contested,Waterway,Brigands
46,Bhalaene,Ghoere,The Heartlands,GT,Hills,High,None,6,1,6,Rook's Roost,True,"[1170, 2085]",False,False,False


In [30]:
regents_i_care_about = ['JR', 'EC', 'GG']
Regent = 'WM'
temp = Game.Regents.copy()
temp = pd.concat([temp[temp['Regent']==a] for a in regents_i_care_about])
temp = pd.concat([temp, Game.Regents[Game.Regents['Regent']==Regent]])

temp

,Regent,Full Name,Player,Class,Level,Alignment,Race,Str,Dex,Con,...,Wis,Cha,Insight,Deception,Persuasion,Regency Points,Gold Bars,Regency Bonus,Attitude,Alive
8,JR,Jaison Raenech,False,Fighter,7,LE,Human,0,1,0,...,2,3,4,5,5,316,7,3,Aggressive,True
0,EC,Eluvie Cariele,False,Fighter,2,CG,Human,0,1,0,...,2,3,4,5,5,316,12,3,Peaceful,True
14,GG,Overthane Grimm Graybeard,False,Fighter/Cleric,2,LG,Dwarf,0,1,0,...,2,3,4,5,5,421,-5,3,Normal,True
9,WM,Willaim Moergen,False,Fighter,5,CG,Human,0,1,0,...,2,3,4,5,5,66,0,3,Normal,True


In [51]:
enemy = 'JR'
import numpy as np
self = Game

temp = Game.Provences[Game.Provences['Regent'] == enemy]
temp_ = pd.concat([self.Troops[self.Troops['Regent']==a] for a in regents_i_care_about])
temp_ = temp_[temp_['Regent'] != enemy]
temp_ = pd.merge(temp[['Provence']], temp_, on='Provence', how='left').fillna(0)
temp_ = temp_[temp_['Type'].astype(str) != '0'][['Provence', 'Type']].groupby('Provence').count().reset_index()
temp_['Occupying Troops'] = temp_['Type']
temp = pd.merge(temp, temp_[['Provence', 'Occupying Troops']], on='Provence', how='left').fillna(0)

temp_ = np.sum(1.0*(temp['Castle']>temp['Occupying Troops'])) #un-neutralized
temp_


0.0